# A full training dataset with the following attributes:

#1: id: unique id for a news article

#2: title: the title of a news article

#3: author: author of the news article

#4: text: the text of the article; could be incomplete

#5: label: a label that marks the article as potentially unreliable


1: Fake news

0: Real news

In [1]:
# importing the dependencies
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shiva\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#printing the stopwords in the English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [26]:
# importing the datasets file
dataset = pd.read_csv("train.csv",encoding=None,encoding_errors="replace",engine = "python")

In [27]:
dataset.head()

,id,title,author,text,label,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 676,Unnamed: 677,Unnamed: 678,Unnamed: 679,Unnamed: 680,Unnamed: 681,Unnamed: 682,Unnamed: 683,Unnamed: 684,Unnamed: 685
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
new_dataset = dataset.drop(dataset.iloc[:,5:],axis=1)

In [29]:
new_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [30]:
# Finding the rows and columns in the dataframe
new_dataset.shape

(25116, 5)

In [31]:
#find the null values in the dataset
new_dataset.isnull().sum()

id           3
title     2250
author    4542
text      3079
label     3446
dtype: int64

In [32]:
# replacing the null values with empty string
new_dataset = new_dataset.fillna('')

In [33]:
#Merge the author data and title data together
new_dataset['Content']= new_dataset['author']+' '+new_dataset['title']

In [34]:
print(new_dataset['Content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
25111    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
25112    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
25113    Michael J. de la Merced and Rachel Abrams Macy...
25114    Alex Ansary NATO, Russia To Hold Parallel Exer...
25115              David Swanson What Keeps the F-35 Alive
Name: Content, Length: 25116, dtype: object


In [35]:
# separating the data & label
x = new_dataset.drop(columns = 'label',axis=1)
y = new_dataset['label']

In [14]:
print(x)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didnât Even See Comeyâs...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
25111  20795  Rapper T.I.: Trump a âPoster Child For White...   
25112  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
25113  20797  Macyâs Is Said to Receive Takeover Approach ...   
25114  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
25115  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [40]:
new_dataset.isnull().sum()

id         0
title      0
author     0
text       0
label      0
Content    0
dtype: int64

# Stemming:

Stemming is the process of reducing a word to its Root word

example: actor, actress, acting --> act

In [37]:
port_stem = PorterStemmer()

In [38]:
def stemming(Content):
    stemmed_Content = re.sub('[^a-zA-Z]',' ',Content)
    stemmed_Content = stemmed_Content.lower()
    stemmed_Content = stemmed_Content.split()
    stemmed_Content = [port_stem.stem(word) for word in stemmed_Content if not word in stopwords.words('english')]
    stemmed_Content = " ".join(stemmed_Content)
    return stemmed_Content


In [39]:
new_dataset['Content'] = new_dataset['Content'].apply(stemming)

In [41]:
print(new_dataset['Content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
25111    jerom hudson rapper trump poster child white s...
25112    benjamin hoffman n f l playoff schedul matchup...
25113    michael j de la merc rachel abram maci said re...
25114    alex ansari nato russia hold parallel exercis ...
25115                            david swanson keep f aliv
Name: Content, Length: 25116, dtype: object


In [42]:
#separating the data and label
x= new_dataset['Content'].values
y= new_dataset['label'].values

In [43]:
print(x)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [44]:
print(y)

['1' '0' '1' ... '0' '1' '1']


In [45]:
y.shape

(25116,)

In [46]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()

In [47]:
vectorizer.fit(x)

TfidfVectorizer()

In [48]:
x = vectorizer.transform(x)

In [49]:
print(x)

  (0, 17070)	0.2880854334390338
  (0, 14648)	0.24170537647240828
  (0, 9694)	0.36554380957366406
  (0, 9387)	0.29316509142349945
  (0, 8387)	0.25144197300521864
  (0, 7617)	0.21896464665093707
  (0, 5380)	0.22421109802197142
  (0, 4092)	0.27191930392178987
  (0, 3884)	0.3619018771847952
  (0, 3190)	0.2502144416754508
  (0, 2676)	0.36953309514559163
  (0, 294)	0.27111435953048824
  (1, 18279)	0.3003541536609625
  (1, 7407)	0.19377544595906765
  (1, 5954)	0.7141085899028712
  (1, 3850)	0.2655601620053946
  (1, 3036)	0.19465794276347997
  (1, 2405)	0.38195552426594254
  (1, 2043)	0.1603300078933037
  (1, 1617)	0.286840953274327
  (2, 16990)	0.4150660858123297
  (2, 10451)	0.4757873319489284
  (2, 6467)	0.3373308801281171
  (2, 5829)	0.3938520403026429
  (2, 3349)	0.4735061475862881
  :	:
  (25113, 14266)	0.23506540036371606
  (25113, 13422)	0.2660285928159704
  (25113, 13201)	0.2509263052625156
  (25113, 11195)	0.08589792606878598
  (25113, 10414)	0.17825077864203123
  (25113, 10332)	0.29

In [50]:
# Spliting the training data and test data
x_train,x_test, y_train,y_test = train_test_split(x,y, test_size = 0.2, stratify = y, random_state= 2)


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
#Training the Model: Logistic Regression
model = LogisticRegression()

In [ ]:
model.fit(x_train, y_train)

# Evaluation

accuracy score

In [ ]:
# accuracy score on the training data
x_train_predication = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_predication, y_train)

print('Accuracy score in the training dataset :',training_data_accuracy)

In [ ]:
# accuracy score on the test data
x_test_predication = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_predication, y_test)

print('Accuracy score in the test dataset :',test_data_accuracy)

# Making a Predictive System

In [ ]:
x_new = x_test[5]

prediction = model.predict(x_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')